In [1]:
from utils_classification import *

# Load data

In [2]:
filepath_suits = os.path.join('data', 'classification_data', 'all_suits.pickle')
segmented = pd.read_pickle(filepath_suits)
values = segmented.values
imgs = [v[1] for v in values]
ranks = [v[0][0] for v in values]
suits = [v[0][1] for v in values]

df_suits = pd.DataFrame({"rank": ranks, "suit": suits, "image": imgs})

In [3]:
df_suits

,rank,suit,image
0,Q,S,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,Q,S,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,8,H,"[[0.0, 0.0, 0.0, 0.014767156862745413, 0.10263..."
3,8,H,"[[0.0, 0.0018457382953180593, 0.08874549819927..."
4,J,H,"[[0.0, 0.0, 0.0011379551820727707, 0.041544117..."
...,...,...,...
723,5,C,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
724,2,H,"[[0.0, 0.0, 0.001395558223289321, 0.0990896358..."
725,2,H,"[[0.0, 0.0, 0.0, 0.0018907563025208232, 0.0672..."
726,6,H,"[[0.0, 0.0, 0.030687274909964084, 0.2806897759..."


## Augment dataset of suits

In [4]:
transform = transforms.Compose(
            [
                transforms.ToPILImage(),
                transforms.RandomAffine(degrees=25, scale=(0.7, 1), shear=25),
                transforms.ToTensor(),
            ]
        )

labels_figures = {'C': 0, 'D': 1, 'H': 2, 'S': 3}

train_augmented_suits, train_augmented_suits_labels, val_augmented_suits, val_augmented_suits_labels, \
            test_suits, test_suits_labels = \
            get_train_val_test_suits(df_suits, labels_figures, transform)

## Data loader

In [5]:
# Parameters of our DataLoader
params = {'batch_size': 128,
          'shuffle': True}

# Creation of a train/test dataset & dataloader 
ds_train = MNISTDataset(train_augmented_suits, train_augmented_suits_labels, transform=transforms.ToTensor())
dl_train = data.DataLoader(ds_train, **params)

ds_val = MNISTDataset(val_augmented_suits, val_augmented_suits_labels, transform=transforms.ToTensor())
dl_val = data.DataLoader(ds_val, **params)

ds_test = MNISTDataset(test_suits, test_suits_labels, transform=transforms.ToTensor())
dl_test = data.DataLoader(ds_test, **params)

# Training and testing

## Models

In [7]:
model = Net(nb_classes=4)

## Training

In [8]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
learning_rate = 1e-3
opt = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
epochs = 10

# Train & test our model until convergence
for e in range(epochs):
    print(f"Epoch {e+1}\n-------------------------------")
    train_loop(dl_train, model, criterion, opt)
    test_loop(dl_val, model, criterion)

Epoch 1
-------------------------------
It 30/30:	Loss train: 0.32101, Accuracy train: 89.41%
Test Error:
	Avg loss: 0.00168, Accuracy: 94.32%

Epoch 2
-------------------------------
It 30/30:	Loss train: 0.04839, Accuracy train: 98.82%
Test Error:
	Avg loss: 0.00026, Accuracy: 100.00%

Epoch 3
-------------------------------
It 30/30:	Loss train: 0.02327, Accuracy train: 100.00%
Test Error:
	Avg loss: 0.00016, Accuracy: 100.00%

Epoch 4
-------------------------------
It 30/30:	Loss train: 0.08096, Accuracy train: 97.65%%
Test Error:
	Avg loss: 0.00012, Accuracy: 100.00%

Epoch 5
-------------------------------
It 30/30:	Loss train: 0.01632, Accuracy train: 100.00%
Test Error:
	Avg loss: 0.00009, Accuracy: 100.00%

Epoch 6
-------------------------------
It 30/30:	Loss train: 0.07442, Accuracy train: 98.82%%
Test Error:
	Avg loss: 0.00011, Accuracy: 100.00%

Epoch 7
-------------------------------
It 30/30:	Loss train: 0.00631, Accuracy train: 100.00%
Test Error:
	Avg loss: 0.00028, 

In [10]:
test_loop(dl_test, model, criterion)

Test Error:
	Avg loss: 0.00116, Accuracy: 98.64%



## Save model and predict

In [11]:
models_dir = os.path.join('data', 'models')

if not os.path.isdir(models_dir):
    os.mkdir(models_dir)

In [12]:
torch.save(model.state_dict(), os.path.join(models_dir, "model_suits.pt"))

In [7]:
predict(test_suits, 4, Net, "data/models/model_suits.pt")

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3])